#  Track Analysis

In order to properly analyze our GPX tracks, we need to put all of them into the same _frame-of-reference_.   This is accomplished by creating a single route, then comparing all routes against that.

## Step 0: Globals

In [1]:
database_path = 'bike_data.db'

start_coord = (39.5989743, -104.8609468)
end_coord   = (39.75428108249532, -105.00085402872664)

epsg_code = 32613

## Step 1: Import Required Libraries

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from pyproj import CRS, Proj, Transformer
import re, numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib as mpl
from matplotlib import cm

### Step 2: Setup the database.

In [3]:
# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT datasetId FROM point_list', conn)

#  Load the Vertex List
vtx_list = pd.read_csv('waypoints.csv')
display(vtx_list)

FileNotFoundError: [Errno 2] No such file or directory: 'waypoints.csv'

### Step 3: Setup UTM Coordinate Projection API

In [ ]:
#  Setup the Projection Transformer
crs = CRS.from_epsg( epsg_code )
proj_dd2utm = Transformer.from_crs(crs.geodetic_crs, crs)
proj_utm2dd = Transformer.from_crs(crs, crs.geodetic_crs)
utm_zone = int(re.findall("\d+", crs.utm_zone)[0])
print('UTM Grid Zone: {}'.format(utm_zone))

(easting,northing) = proj_dd2utm.transform( start_coord[0], start_coord[1] )
start_coord_utm = np.array( [easting, northing], np.float64 )

## Step 4: Visualize "Best-Fit" Routes from the GA

In [ ]:
mapbox_access_token = open(".mapbox_token").read()

fig = go.Figure()

#  Add Points
sql_query = 'SELECT * FROM point_list ORDER BY timestamp'
full_points = pd.read_sql_query( sql_query, conn )
fig.add_trace(go.Scattermapbox(
        lat=full_points.loc[:,'latitude'],
        lon=full_points.loc[:,'longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker( size=5,
                                        color='rgb(255, 0, 0)',
                                        opacity=0.7 )
    ))

centroid = [full_points.loc[:,'latitude'].mean(),
            full_points.loc[:,'longitude'].mean()]
fig.update_layout(
    title='Coordinate Information',
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=centroid[0],
            lon=centroid[1]
        ),
        pitch=0,
        zoom=13,
        style='light'
    ),
    height=700,
)

#  Get the list of waypoint sizes
waypoint_sizes = vtx_list['NumWaypoints'].unique()
colors = {}
waypoints = {}
cmap = mpl.cm.get_cmap('Spectral')
for wp in waypoint_sizes:
    colors[wp] = 0
    print()
    if len(waypoint_sizes) > 0 or (waypoint_sizes.max() - waypoint_sizes.min()) < 0.1:
        colors[wp] = cmap((wp - waypoint_sizes.min()) / (waypoint_sizes.max() - waypoint_sizes.min()))
    waypoints[wp] = [[],[],[]]
for wp in vtx_list.itertuples():
    waypoints[wp.NumWaypoints][0].append( wp.Latitude )
    waypoints[wp.NumWaypoints][1].append( wp.Longitude )
    waypoints[wp.NumWaypoints][2].append( str(wp) + ' Waypoints' )
    
for wp in waypoints:
    fig.add_trace(go.Scattermapbox(
        lat=waypoints[wp][0],
        lon=waypoints[wp][1],
        mode='markers+lines',
        line=dict(width=4),
        text=waypoints[wp][2],
        marker=go.scattermapbox.Marker( size=5,
                                        color=colors[wp],
                                        opacity=0.7 )
    ))
fig.show()

## Step 6: Iterate through each sector, constructing the path

In [ ]:
# Get Sector List
sql_query = 'SELECT * FROM sector_list'
sector_table = pd.read_sql_query( sql_query, conn )

# Show the map


e_max_vals = { 'sector_0': 50,
               'sector_1': 327.708,
               'sector_2': 50,
               'sector_3': 50,
               'sector_4': 50,
               'sector_5': 50,
               'sector_6': 50,
               'sector_7': 50,
               'sector_8': 50 }
    
fig.show()